# Pymol

In [25]:
import pymol
from pymol import cmd
import pandas as pd

# --- Load scores ---
df = pd.read_csv("site_scores.csv")   # must have columns: i, escapemap
score_dict = dict(zip(df["i"].astype(int), df["escapemap"].astype(float)))
sites = list(score_dict.keys())

# Start PyMOL
pymol.finish_launching()

# Fetch PDB
pdb_id = "6M0J"
cmd.fetch(pdb_id, "6M0J")

# Background
cmd.bg_color("white")
cmd.hide("everything", "all")

# --- ACE2 (chain A) always shown in dark blue ---
cmd.show("cartoon", "chain A")
cmd.color("blue", "chain A")

# --- Spike (chain E): only show residues that are in CSV ---
cmd.hide("cartoon", "chain E")  # hide all spike
sel_scored = "chain E and resi " + "+".join(map(str, sites))
cmd.show("cartoon", sel_scored)

# Assign scores into b-factor
for resi, score in score_dict.items():
    cmd.alter(f"chain E and resi {resi}", f"b={score}")

# Normalize to min–max range for strong contrast
all_scores = list(score_dict.values())
min_score, max_score = min(all_scores), max(all_scores)

cmd.spectrum("b", "blue_white_red", sel_scored,
             minimum=min_score, maximum=max_score)

cmd.zoom("chain E", buffer=5)

cmd.set("ray_opaque_background", 0)
cmd.ray(1920, 1080)

print("Only Spike residues in CSV are shown and colored from blue→white→red. Others are hidden.")
